# Deploy a model to online endpoints using Triton
Learn how to deploy a model using Triton as an online endpoint in Azure Machine Learning.

Triton is multi-framework, open-source software that is optimized for inference. It supports popular machine learning frameworks like TensorFlow, ONNX Runtime, PyTorch, NVIDIA TensorRT, and more. It can be used for your CPU or GPU workloads.

## Prerequisites

* To use Azure Machine Learning, you must have an Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/).

* Install and configure the [Python SDK v2](sdk/setup.sh).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace. 

### Please note, for Triton no-code-deployment, testing via local endpoints is currently not supported, so this tutorial will only show how to set up on online endpoint.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

In [50]:
# Import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, AzureCliCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [51]:
# enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [52]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace # Restore to default
)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FDRCQDFXU to authenticate.
^C


# 2. Install Additional Requirements

Install additional Python requirements using the following command. These will be used for scoring.

`pip install numpy tritonclient[http] pillow gevent`

# 3. Deploy your online endpoint to Azure
Next, deploy your online endpoint to Azure.

## 3.1 Configure online endpoint
`endpoint_name`: The name of the endpoint. It must be unique in the Azure region. Naming rules are defined under [managed online endpoint limits](https://docs.microsoft.com/azure/machine-learning/how-to-manage-quotas#azure-machine-learning-managed-online-endpoints-preview).

`auth_mode` : Use `key` for key-based authentication. Use `aml_token` for Azure Machine Learning token-based authentication. A `key` does not expire, but `aml_token` does expire. 

Optionally, you can add description, tags to your endpoint.

In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)

## 3.2 Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [ ]:
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08261710825185.westus2.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-08261710825185.westus2.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08261710825185', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourcegroups/rg-azureml-pg/providers/microsoft.machinelearningservices/workspaces/aml-pg-01/onlineendpoints/endpoint-08261710825185', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/providers/Microsoft.MachineLearningServices/locations/westus2/mfeOperationsStatus/oe:fb0892ae-a828-4c36-a8cb-ce6ad3c8a0fd:e1496590-f7e6-4fd1-ae7a-f35ec177d58e?api-version=2022-02-01-preview'}, 'id': '/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourceGroups/rg-azurem

## 3.3 Configure online deployment
A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class.

### Key aspects of deployment 
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_configuration` - the configuration for the source code and scoring script
    - `path`- Path to the source code directory for scoring the model
    - `scoring_script` - Relative path to the scoring file in the source code directory
- `instance_type` - The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](https://docs.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list).
- `instance_count` - The number of instances to use for the deployment

In [ ]:
# create a blue deployment
model = Model(name="sample-densenet-onnx-model", version="1", path="./models", type="triton_model")

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_NC6s_v3",
    instance_count=1,
)

### Readiness route vs. liveness route
An HTTP server defines paths for both liveness and readiness. A liveness route is used to check whether the server is running. A readiness route is used to check whether the server is ready to do work. In machine learning inference, a server could respond 200 OK to a liveness request before loading a model. The server could respond 200 OK to a readiness request only after the model has been loaded into memory.

Review the [Kubernetes documentation](https://kubernetes.io/docs/tasks/configure-pod-container/configure-liveness-readiness-startup-probes/) for more information about liveness and readiness probes.

Notice that this deployment uses the same path for both liveness and readiness, since TF Serving only defines a liveness route.

## 3.4 Create the deployment
Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.begin_create_or_update(blue_deployment)

Check: endpoint endpoint-08261710825185 exists
Uploading models (32.72 MBs): 100%|██████████| 32719461/32719461 [00:00<00:00, 46065399.42it/s]


Creating/updating online deployment blue 

...............................................................................................................

Done (9m 51s)


## 3.4 Set traffic to 100% for deployment

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08261710825185.westus2.inference.ml.azure.com/', 'swagger_uri': 'https://endpoint-08261710825185.westus2.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08261710825185', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourcegroups/rg-azureml-pg/providers/microsoft.machinelearningservices/workspaces/aml-pg-01/onlineendpoints/endpoint-08261710825185', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/providers/Microsoft.MachineLearningServices/locations/westus2/mfeOperationsStatus/oe:fb0892ae-a828-4c36-a8cb-ce6ad3c8a0fd:cbb71e63-ba82-41b8-889f-44b1bfa5effb?api-version=2022-02-01-preview'}, 'id': '/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourceGroups/rg-azureml-pg/

# 4. Test the endpoint with sample data
This version of the triton server requires pre- and post-image processing. Below we show how to invoke the endpoint with this processing.

In [48]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://endpoint-08261710825185.westus2.inference.ml.azure.com/


In [ ]:
# Enter the scoring_url and auth token for your AML deployment

scoring_uri = endpoint.scoring_uri
auth_token = "<AUTH_TOKEN"
img_url = "https://aka.ms/peacock-pic" # This is a sample image

In [ ]:
# test the blue deployment with some sample data
from scoring_utils import prepost
import requests
import numpy as np

import gevent.ssl
import tritonclient.http as tritonhttpclient

# We remove the scheme from the url
scoring_uri = scoring_uri[8:]

# Initialize client handler 
triton_client = tritonhttpclient.InferenceServerClient(
        url=scoring_uri,
        ssl=True,
        ssl_context_factory=gevent.ssl._create_default_https_context,
    )

# Create headers
headers = {}
headers["Authorization"] = f"Bearer {auth_token}"

# Check status of triton server
health_ctx = triton_client.is_server_ready(headers=headers)
print("Is server ready - {}".format(health_ctx))

# Check status of model
model_name = "model_1"
status_ctx = triton_client.is_model_ready(model_name, "1", headers)
print("Is model ready - {}".format(status_ctx))

img_content = requests.get(img_url).content
img_data = prepost.preprocess(img_content)

# Populate inputs and outputs
input = tritonhttpclient.InferInput("data_0", img_data.shape, "FP32")
input.set_data_from_numpy(img_data)
inputs = [input]
output = tritonhttpclient.InferRequestedOutput("fc6_1")
outputs = [output]

result = triton_client.infer(model_name, inputs, outputs=outputs, headers=headers)
max_label = np.argmax(result.as_numpy("fc6_1"))
label_name = prepost.postprocess(max_label)
print(label_name)

Is server ready - True
Is model ready - True
/data/workspace/dalabrun/azureml-examples/sdk/endpoints/online/triton/scoring_utils/densenet_labels.txt
84 : PEACOCK


# 5. Managing endpoints and deployments

## 5.1 Get the logs for the new deployment
Get the logs for the blue deployment and verify as needed

In [53]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

"Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Downloading, Type: Normal, Time: 2022-08-26T17:17:43.099773Z, Message: Start downloading models\nKind: Pod, Name: Pulling, Type: Normal, Time: 2022-08-26T17:17:48.443422Z, Message: Start pulling container image\nKind: Pod, Name: Pulled, Type: Normal, Time: 2022-08-26T17:22:00.421567Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2022-08-26T17:22:00.421567Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2022-08-26T17:22:00.457065Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2022-08-26T17:22:00.628379Z, Message: Started container inference-server\nKind: Pod, Name: ContainerReady, Type: Normal, Time: 2022-08-26T17:22:19.82993125Z, Message: Container is ready\n\nContainer logs

# 6. Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)